<a href="https://colab.research.google.com/github/MooNight009/ColabProjects/blob/master/DONE_imdb_text_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
%tensorflow_version 2.x

from __future__ import absolute_import, division, print_function, unicode_literals

import numpy as np

import tensorflow as tf
tf.executing_eagerly
!pip install -q tensorflow-hub
!pip install -q tensorflow-datasets
import tensorflow_hub as hub
import tensorflow_datasets as tfds

print("Version: ", tf.__version__)
print("Eager mode: ", tf.executing_eagerly())
print("Hub version: ", hub.__version__)
print("GPU is", "available" if tf.config.experimental.list_physical_devices("GPU") else "NOT AVAILABLE")

TensorFlow 2.x selected.
Version:  2.1.0-rc1
Eager mode:  True
Hub version:  0.7.0
GPU is available


In [0]:
# Split the training set into 60% and 40%, so we'll end up with 15,000 examples
# for training, 10,000 examples for validation and 25,000 examples for testing.
train_validation_split = tfds.Split.TRAIN.subsplit([6, 4])

(train_data, validation_data), test_data = tfds.load(
    name="imdb_reviews", 
    split=(train_validation_split, tfds.Split.TEST),
    as_supervised=True)

In [0]:
train_examples_batch, train_labels_batch = next(iter(train_data.batch(10)))
train_examples_batch

In [0]:
train_labels_batch

In [0]:
embedding = "https://tfhub.dev/google/tf2-preview/gnews-swivel-20dim/1"
hub_layer = hub.KerasLayer(embedding, input_shape=[], 
                           dtype=tf.string, trainable=True)
hub_layer(train_examples_batch[:3])

In [0]:
model = tf.keras.Sequential()
model.add(hub_layer)
model.add(tf.keras.layers.Dense(14, activation='relu'))
model.add(tf.keras.layers.Dense(1, activation='sigmoid'))

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
keras_layer (KerasLayer)     (None, 20)                400020    
_________________________________________________________________
dense (Dense)                (None, 14)                294       
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 15        
Total params: 400,329
Trainable params: 400,329
Non-trainable params: 0
_________________________________________________________________


In [0]:
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [0]:
history = model.fit(train_data.shuffle(10000).batch(512),
                    epochs=30,
                    validation_data=validation_data.batch(512),
                    verbose=1)

Epoch 1/50
30/30 [==============================] - 6s 191ms/step - loss: 0.9470 - accuracy: 0.4480 - val_loss: 0.7639 - val_accuracy: 0.4815
Epoch 2/50
30/30 [==============================] - 5s 151ms/step - loss: 0.6974 - accuracy: 0.5557 - val_loss: 0.6616 - val_accuracy: 0.6081
Epoch 3/50
30/30 [==============================] - 5s 152ms/step - loss: 0.6425 - accuracy: 0.6367 - val_loss: 0.6288 - val_accuracy: 0.6496
Epoch 4/50
30/30 [==============================] - 5s 156ms/step - loss: 0.6106 - accuracy: 0.6709 - val_loss: 0.6020 - val_accuracy: 0.6749
Epoch 5/50
30/30 [==============================] - 5s 164ms/step - loss: 0.5805 - accuracy: 0.6981 - val_loss: 0.5750 - val_accuracy: 0.7005
Epoch 6/50
30/30 [==============================] - 5s 160ms/step - loss: 0.5485 - accuracy: 0.7287 - val_loss: 0.5439 - val_accuracy: 0.7310
Epoch 7/50
30/30 [==============================] - 5s 153ms/step - loss: 0.5130 - accuracy: 0.7597 - val_loss: 0.5123 - val_accuracy: 0.7560
Epoch 

In [0]:
results = model.evaluate(test_data.batch(512), verbose=2)

for name, value in zip(model.metrics_names, results):
  print("%s: %.3f" % (name, value))

loss: 0.446
accuracy: 0.853
